In [1]:
import numpy as np

np.random.seed(1)

In [2]:
# channels, width, height
_input_img = np.random.rand(2,3,4)
_input_size = _input_img.shape

#input_img[x][y][z] += 100
_input_img

array([[[4.17022005e-01, 7.20324493e-01, 1.14374817e-04, 3.02332573e-01],
        [1.46755891e-01, 9.23385948e-02, 1.86260211e-01, 3.45560727e-01],
        [3.96767474e-01, 5.38816734e-01, 4.19194514e-01, 6.85219500e-01]],

       [[2.04452250e-01, 8.78117436e-01, 2.73875932e-02, 6.70467510e-01],
        [4.17304802e-01, 5.58689828e-01, 1.40386939e-01, 1.98101489e-01],
        [8.00744569e-01, 9.68261576e-01, 3.13424178e-01, 6.92322616e-01]]])

In [3]:
_num_kernels = 3
_kernel_size = 3
_kernels = np.random.rand(_num_kernels,_kernel_size,_kernel_size)
_kernels

array([[[0.87638915, 0.89460666, 0.08504421],
        [0.03905478, 0.16983042, 0.8781425 ],
        [0.09834683, 0.42110763, 0.95788953]],

       [[0.53316528, 0.69187711, 0.31551563],
        [0.68650093, 0.83462567, 0.01828828],
        [0.75014431, 0.98886109, 0.74816565]],

       [[0.28044399, 0.78927933, 0.10322601],
        [0.44789353, 0.9085955 , 0.29361415],
        [0.28777534, 0.13002857, 0.01936696]]])

In [4]:
def apply_1d_padding(input_img, kernel_size):
    pad = kernel_size//2
    return np.concatenate([np.zeros(pad), input_img, np.zeros(pad)])

def apply_2d_padding(input_img, kernel_size):
    padded = []
    for channel in input_img:
        pad_side = np.stack([apply_1d_padding(row, kernel_size) for row in channel])
        width = pad_side.shape[1]
        pad_full = np.vstack([np.zeros(width), pad_side,np.zeros(width)])
        padded.append(pad_full)
    return np.stack(padded)

#_input_pad = apply_2d_padding(_input_img, _kernel_size)
#_input_pad

array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 4.17022005e-01, 7.20324493e-01, 1.14374817e-04,
         3.02332573e-01, 0.00000000e+00],
        [0.00000000e+00, 1.46755891e-01, 9.23385948e-02, 1.86260211e-01,
         3.45560727e-01, 0.00000000e+00],
        [0.00000000e+00, 3.96767474e-01, 5.38816734e-01, 4.19194514e-01,
         6.85219500e-01, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.04452250e-01, 8.78117436e-01, 2.73875932e-02,
         6.70467510e-01, 0.00000000e+00],
        [0.00000000e+00, 4.17304802e-01, 5.58689828e-01, 1.40386939e-01,
         1.98101489e-01, 0.00000000e+00],
        [0.00000000e+00, 8.00744569e-01, 9.68261576e-01, 3.13424178e-01,
     

In [5]:
# Forward pass
def forward_propagation(input_img, kernels, num_kernels, kernel_size, input_size):
    output = np.zeros(shape=(num_kernels, input_size[1], input_size[2]))
    thing = "hello"
    input_pad = apply_2d_padding(input_img, kernel_size)

    for i in range(input_img.shape[0]): # input channels
        for k in range(num_kernels): # output channels
            for i_w in range(input_img.shape[1]): # img width
                for i_h in range(input_img.shape[2]): # img height
                    for k_w in range(kernel_size):
                        for k_h in range(kernel_size):
                            output[k][i_w][i_h] += kernels[k][k_w][k_h] * input_pad[i][i_w+k_w][i_h+k_h]
    return output

_output = forward_propagation(_input_img, _kernels, _num_kernels, _kernel_size, _input_size)
_output

array([[[2.3703486 , 0.96240475, 1.64370311, 0.42735015],
        [3.30729962, 3.85058689, 3.84276579, 1.6516076 ],
        [2.08678215, 2.05059042, 2.30196813, 1.03519468]],

       [[1.59278589, 3.07253606, 2.35619988, 1.61344296],
        [3.72872024, 5.31923632, 4.79331176, 3.27748677],
        [1.62270649, 2.94756924, 2.41590527, 2.20297979]],

       [[1.11994633, 1.99206763, 1.2669003 , 1.06089235],
        [1.54407003, 2.93357994, 1.87404704, 1.80574537],
        [2.04295808, 2.82653887, 2.24164346, 2.10047132]]])

In [24]:
# Backward Pass
def backward_propagation_input(input_img, kernels, num_kernels, kernel_size, output, output_grad, learning_rate):
    output_grad_pad = apply_2d_padding(output_grad, kernel_size)

    input_grad = np.zeros_like(input_img)    

    for i in range(input_img.shape[0]): # input channels
        for k in range(num_kernels): # output channels
            for i_w in range(input_img.shape[1]): # img width
                for i_h in range(input_img.shape[2]): # img height
                    for k_w in range(kernel_size):
                        for k_h in range(kernel_size):
                            input_grad[i][i_w][i_h] += output_grad_pad[k][i_w+kernel_size-k_w-1][i_h+kernel_size-k_h-1] * kernels[k][k_w][k_h] # or i

    return input_grad

def backward_propagation_kernels(input_img, kernels, num_kernels, kernel_size, output, output_grad, learning_rate):
    # kernel gradient
    kernels_grad = np.zeros_like(kernels)

    input_pad = apply_2d_padding(input_img, kernel_size)
    
    for i in range(input_img.shape[0]): # input channels
        for k in range(num_kernels): # output channels
            for i_w in range(input_img.shape[1]): # img width
                for i_h in range(input_img.shape[2]): # img height
                    for k_w in range(kernel_size):
                        for k_h in range(kernel_size):
                            kernels_grad[k][k_w][k_h] += input_pad[i][i_w+k_w][i_h+k_h] * output_grad[k][k_w][k_h]

    return kernels_grad

#backward_propagation_input(_output, _output, 0.001)

In [25]:
# Test gradients
_input_img2 = _input_img.copy()
x = np.random.randint(_input_img.shape[0])
y = np.random.randint(_input_img.shape[1])
z = np.random.randint(_input_img.shape[2])
print((x,y,z))
_input_img2[x][y][z] += 1
#print(_input_img)
#print(_input_img2)

_output = forward_propagation(_input_img, _kernels, _num_kernels, _kernel_size, _input_size)
_output2 = forward_propagation(_input_img2, _kernels, _num_kernels, _kernel_size, _input_size)

#print(_output - _output2)
s1 = _output.sum()
s2 = _output2.sum()
print(s2-s1)

(1, 2, 2)
8.678231676587544


In [26]:
_output_error = np.ones_like(_output)

_input_grad = backward_propagation_input(_input_img, _kernels, _num_kernels, _kernel_size, _output, _output_error, 0.001)
_input_grad[x][y][z]

8.678231676587522

In [27]:
_kernels2 = _kernels.copy()
x = np.random.randint(_kernels.shape[0])
y = np.random.randint(_kernels.shape[1])
z = np.random.randint(_kernels.shape[2])
_kernels2[x][y][z] += 1

_output = forward_propagation(_input_img, _kernels, _num_kernels, _kernel_size, _input_size)
_output2 = forward_propagation(_input_img, _kernels2, _num_kernels, _kernel_size, _input_size)
s1 = _output.sum()
s2 = _output2.sum()
print(s2-s1)

10.120367878849919


In [28]:
_kernels_grad = backward_propagation_kernels(_input_img, _kernels, _num_kernels, _kernel_size, _output, _output_error, 0.001)
_kernels_grad[x][y][z]

10.1203678788499